<a href="https://colab.research.google.com/github/wallas999/Mis_Proyectos/blob/main/Elevators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Versión 1.2
# 1 elevador, 2 lógicas de operación


#* La función con algoritmo SCAN clásico
#* Cálculo de energía simplificado
#* Visualización del historial con pandas


# Importamos pandas para mostrar resultados ordenados
import pandas as pd
from IPython.display import display, HTML






import pandas as pd

def simular_scan(
    llamadas,
    piso_inicial,
    piso_min=1,            # piso mínimo del edificio
    piso_max=10,           # piso máximo del edificio
    consumo_subida=4.0,    # Wh por piso subido
    consumo_bajada=1.0,    # Wh por piso bajado
    consumo_parada=1.0,    # Wh por parada
    tiempo_por_piso=5,     # segundos por piso recorrido
    tiempo_parada=10       # segundos por parada
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    # Filtrar llamadas arriba y abajo respecto a piso_actual
    llamadas_arriba = sorted([p for p in llamadas if p > piso_actual])
    llamadas_abajo = sorted([p for p in llamadas if p < piso_actual], reverse=True)

    # Agregar piso máximo al recorrido arriba si no está ya en llamadas_arriba
    if piso_max not in llamadas_arriba and piso_actual < piso_max:
        llamadas_arriba.append(piso_max)
        llamadas_arriba = sorted(llamadas_arriba)

    # Agregar piso mínimo al recorrido abajo si no está ya en llamadas_abajo
    if piso_min not in llamadas_abajo and piso_actual > piso_min:
        llamadas_abajo.append(piso_min)
        llamadas_abajo = sorted(llamadas_abajo, reverse=True)

    # Atender llamadas hacia arriba (incluyendo el piso máximo)
    for piso_destino in llamadas_arriba:
        pisos = abs(piso_destino - piso_actual)
        energia_mov = pisos * consumo_subida
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa

        tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual,
            "Hasta": piso_destino,
            "Sube": True,
            "Distancia (pisos)": pisos,
            "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total,
            "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = piso_destino

    # Atender llamadas hacia abajo (incluyendo el piso mínimo)
    for piso_destino in llamadas_abajo:
        pisos = abs(piso_destino - piso_actual)
        energia_mov = pisos * consumo_bajada
        energia_etapa = energia_mov + consumo_parada
        consumo_total += energia_etapa

        tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
        tiempo_total += tiempo_etapa

        historial.append({
            "Desde": piso_actual,
            "Hasta": piso_destino,
            "Sube": False,
            "Distancia (pisos)": pisos,
            "Energía etapa (Wh)": energia_etapa,
            "Energía acumulada (Wh)": consumo_total,
            "Tiempo etapa (s)": tiempo_etapa,
            "Tiempo acumulado (s)": tiempo_total
        })
        piso_actual = piso_destino

    return pd.DataFrame(historial), consumo_total, tiempo_total


def simular_look(
    llamadas,
    piso_inicial,
    consumo_subida=4.0,     # Wh por piso subido
    consumo_bajada=1.0,     # Wh por piso bajado
    consumo_parada=1.0,     # Wh por parada
    tiempo_por_piso=5,      # segundos por piso recorrido
    tiempo_parada=10        # segundos por parada
):
    piso_actual = piso_inicial
    consumo_total = 0.0
    tiempo_total = 0
    historial = []

    # Dividir llamadas en arriba y abajo respecto al piso actual
    llamadas_arriba = sorted([p for p in llamadas if p > piso_actual])
    llamadas_abajo = sorted([p for p in llamadas if p < piso_actual], reverse=True)

    # Primero atender llamadas arriba, solo hasta la última llamada arriba
    if llamadas_arriba:
        for piso_destino in llamadas_arriba:
            pisos = abs(piso_destino - piso_actual)
            energia_mov = pisos * consumo_subida
            energia_etapa = energia_mov + consumo_parada
            consumo_total += energia_etapa

            tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
            tiempo_total += tiempo_etapa

            historial.append({
                "Desde": piso_actual,
                "Hasta": piso_destino,
                "Sube": True,
                "Distancia (pisos)": pisos,
                "Energía etapa (Wh)": energia_etapa,
                "Energía acumulada (Wh)": consumo_total,
                "Tiempo etapa (s)": tiempo_etapa,
                "Tiempo acumulado (s)": tiempo_total
            })
            piso_actual = piso_destino

    # Luego atender llamadas abajo, solo hasta la última llamada abajo
    if llamadas_abajo:
        for piso_destino in llamadas_abajo:
            pisos = abs(piso_destino - piso_actual)
            energia_mov = pisos * consumo_bajada
            energia_etapa = energia_mov + consumo_parada
            consumo_total += energia_etapa

            tiempo_etapa = pisos * tiempo_por_piso + tiempo_parada
            tiempo_total += tiempo_etapa

            historial.append({
                "Desde": piso_actual,
                "Hasta": piso_destino,
                "Sube": False,
                "Distancia (pisos)": pisos,
                "Energía etapa (Wh)": energia_etapa,
                "Energía acumulada (Wh)": consumo_total,
                "Tiempo etapa (s)": tiempo_etapa,
                "Tiempo acumulado (s)": tiempo_total
            })
            piso_actual = piso_destino

    return pd.DataFrame(historial), consumo_total, tiempo_total


In [3]:
#EJECUTAR AMBAS y PLOTEAR

# --- Supongamos que ya tenís definidas las funciones simular_scan y simular_look ---
# Por ejemplo:
# df_historial_scan, total_energia_scan, total_tiempo_scan = simular_scan(llamadas, piso_inicial)
# df_historial_look, total_energia_look, total_tiempo_look = simular_look(llamadas, piso_inicial)

# Parámetros de prueba
piso_inicial = 3
llamadas = [2, 8, 4, 6]

# Ejecutar simulación SCAN
df_historial_scan, total_energia_scan, total_tiempo_scan = simular_scan(llamadas, piso_inicial)
# Crear lista de pisos recorridos para SCAN
pisos_recorrido_scan = [piso_inicial] + df_historial_scan["Hasta"].tolist()

# Ejecutar simulación LOOK
df_historial_look, total_energia_look, total_tiempo_look = simular_look(llamadas, piso_inicial)
# Crear lista de pisos recorridos para LOOK
pisos_recorrido_look = [piso_inicial] + df_historial_look["Hasta"].tolist()

# Ahora podís usar esos datos en la animación que te pasé antes,
# donde se dibujan dos animaciones lado a lado para comparar.

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Determinar el rango común de pisos para ambas simulaciones
piso_min = min(min(pisos_recorrido_scan), min(pisos_recorrido_look))
piso_max = max(max(pisos_recorrido_scan), max(pisos_recorrido_look))

plt.ioff()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 4), sharey=True)

for ax in (ax1, ax2):
    ax.set_ylim(piso_min - 1, piso_max + 1)
    ax.set_xlim(0, 1)
    ax.set_xticks([])
    ax.set_ylabel("Piso")

ax1.set_title("Elevator SCAN")
ax2.set_title("Elevator LOOK")

for piso in range(piso_min, piso_max + 1):
    ax1.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')
    ax2.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')

punto_scan, = ax1.plot([], [], 's', markersize=20, color='blue')
texto_energia_scan = ax1.text(0.5, piso_min - 1, '', ha='center')

punto_look, = ax2.plot([], [], 's', markersize=20, color='green')
texto_energia_look = ax2.text(0.5, piso_min - 1, '', ha='center')

max_frames = max(len(pisos_recorrido_scan), len(pisos_recorrido_look))

def init():
    punto_scan.set_data([], [])
    texto_energia_scan.set_text('')
    punto_look.set_data([], [])
    texto_energia_look.set_text('')
    return punto_scan, texto_energia_scan, punto_look, texto_energia_look

def actualizar(frame):
    if frame < len(pisos_recorrido_scan):
        piso_s = pisos_recorrido_scan[frame]
        punto_scan.set_data([0.5], [piso_s])
        # resto igual...

    else:
        piso_s = pisos_recorrido_scan[-1]
        punto_scan.set_data([0.5], [piso_s])

    if frame < len(pisos_recorrido_look):
        piso_l = pisos_recorrido_look[frame]
        punto_look.set_data([0.5], [piso_l])
        # resto igual...

    else:
        piso_l = pisos_recorrido_look[-1]
        punto_look.set_data([0.5], [piso_l])

    return punto_scan, texto_energia_scan, punto_look, texto_energia_look


ani = animation.FuncAnimation(
    fig, actualizar, frames=max_frames,
    init_func=init, blit=True, interval=1000, repeat=False
)

HTML(ani.to_jshtml())


In [2]:
#EJECUCIÓN SIMULAR SCAN
# Parámetros de prueba
piso_inicial = 3
llamadas = [2, 8, 4, 6]

# Ejecutar simulación
df_historial, energia_total, tiempo_total = simular_scan(llamadas, piso_inicial)

# Crear copia para modificar nombres de columnas
df_mod = df_historial.copy()

# Cambiar nombres para mostrar saltos de línea con <br>
df_mod.columns = [
    "Desde",
    "Hasta",
    "Sube",
    "Distancia (pisos)",
    "Energía etapa (Wh)",
    "Energía acumulada (Wh)",
    "Tiempo etapa (s)",
    "Tiempo acumulado (s)"
]

# Mostrar DataFrame con encabezados en dos líneas
# display(HTML(df_mod.to_html(escape=False))) #Opción para dar mayor formato a tabla (DataFrame)
display(df_mod)

print(f"\nEnergía total: {energia_total:.2f} Wh")
print(f"Tiempo total: {tiempo_total} segundos")



,Desde,Hasta,Sube,Distancia (pisos),Energía etapa (Wh),Energía acumulada (Wh),Tiempo etapa (s),Tiempo acumulado (s)
0,3,4,True,1,5.0,5.0,15,15
1,4,6,True,2,9.0,14.0,20,35
2,6,8,True,2,9.0,23.0,20,55
3,8,10,True,2,9.0,32.0,20,75
4,10,2,False,8,9.0,41.0,50,125
5,2,1,False,1,2.0,43.0,15,140



Energía total: 43.00 Wh
Tiempo total: 140 segundos


In [3]:
#EJECUCIÓN SIMULAR LOOK
piso_inicial = 3
llamadas = [2, 8, 4, 6]

df_historial_look, energia_look, tiempo_look = simular_look(llamadas, piso_inicial)

display(df_historial_look)
print(f"Energía total LOOK: {energia_look:.2f} Wh")
print(f"Tiempo total LOOK: {tiempo_look} segundos")


,Desde,Hasta,Sube,Distancia (pisos),Energía etapa (Wh),Energía acumulada (Wh),Tiempo etapa (s),Tiempo acumulado (s)
0,3,4,True,1,5.0,5.0,15,15
1,4,6,True,2,9.0,14.0,20,35
2,6,8,True,2,9.0,23.0,20,55
3,8,2,False,6,7.0,30.0,40,95


Energía total LOOK: 30.00 Wh
Tiempo total LOOK: 95 segundos


In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Se asume que las variables correspondientes a cada simulación están definidas:
# df_historial_scan, pisos_recorrido_scan
# df_historial_look, pisos_recorrido_look

# Determinar el piso mínimo y máximo considerando ambos recorridos para unificar escalas
piso_min = min(min(pisos_recorrido_scan), min(pisos_recorrido_look))
piso_max = max(max(pisos_recorrido_scan), max(pisos_recorrido_look))

plt.ioff()  # Desactivar la generación automática de figuras estáticas

# Crear una figura con dos subgráficos en una fila, compartiendo el eje Y
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 4), sharey=True)

# Configuración común para ambos subgráficos
for ax in (ax1, ax2):
    ax.set_ylim(piso_min - 1, piso_max + 1)  # Limitar eje Y según rango de pisos
    ax.set_xlim(0, 1)                       # Limitar eje X para representar solo la posición lateral
    ax.set_xticks([])                       # Ocultar marcas en eje X
    ax.set_ylabel("Piso")                   # Etiqueta del eje Y

# Títulos específicos para cada algoritmo de simulación
ax1.set_title("Elevator SCAN")
ax2.set_title("Elevator LOOK")

# Dibujar líneas horizontales para cada piso en ambos subgráficos
for piso in range(piso_min, piso_max + 1):
    ax1.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')
    ax2.hlines(piso, 0, 1, colors='lightgray', linestyles='dashed')

# Crear objetos gráficos que representan la posición del ascensor y el texto informativo
punto_scan, = ax1.plot([], [], 's', markersize=20, color='blue')
texto_energia_scan = ax1.text(0.5, piso_min - 1, '', ha='center')

punto_look, = ax2.plot([], [], 's', markersize=20, color='green')
texto_energia_look = ax2.text(0.5, piso_min - 1, '', ha='center')

# Determinar el número máximo de cuadros para sincronizar la animación de ambos ascensores
max_frames = max(len(pisos_recorrido_scan), len(pisos_recorrido_look))

def init():
    """Función de inicialización para la animación."""
    punto_scan.set_data([], [])
    texto_energia_scan.set_text('')
    punto_look.set_data([], [])
    texto_energia_look.set_text('')
    return punto_scan, texto_energia_scan, punto_look, texto_energia_look

def actualizar(frame):
    """Función que actualiza la posición y texto de cada ascensor en cada frame."""
    # Actualizar ascensor SCAN
    if frame < len(pisos_recorrido_scan):
        piso_s = pisos_recorrido_scan[frame]
        punto_scan.set_data(0.5, piso_s)
        if frame == 0:
            texto_energia_scan.set_text(f"Piso inicial: {piso_s}")
        else:
            energia_s = df_historial_scan.iloc[frame-1]["Energía etapa (Wh)"]
            texto_energia_scan.set_text(f"Energía etapa:\n{energia_s:.1f} Wh")
    else:
        piso_s = pisos_recorrido_scan[-1]
        punto_scan.set_data(0.5, piso_s)

    # Actualizar ascensor LOOK
    if frame < len(pisos_recorrido_look):
        piso_l = pisos_recorrido_look[frame]
        punto_look.set_data(0.5, piso_l)
        if frame == 0:
            texto_energia_look.set_text(f"Piso inicial: {piso_l}")
        else:
            energia_l = df_historial_look.iloc[frame-1]["Energía etapa (Wh)"]
            texto_energia_look.set_text(f"Energía etapa:\n{energia_l:.1f} Wh")
    else:
        piso_l = pisos_recorrido_look[-1]
        punto_look.set_data(0.5, piso_l)

    return punto_scan, texto_energia_scan, punto_look, texto_energia_look

# Crear la animación combinada para ambas simulaciones
ani = animation.FuncAnimation(
    fig, actualizar, frames=max_frames,
    init_func=init, blit=True, interval=1000, repeat=False
)

# Mostrar la animación en formato HTML compatible con Jupyter/Colab
HTML(ani.to_jshtml())


NameError: name 'pisos_recorrido_scan' is not defined